In [ ]:
### library import 
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import opz data
df_lab_opz = pd.read_pickle('df_lab_opz.pkl')
df_name_opz = pd.read_pickle('df_name_opz.pkl')
df_vital_opz = pd.read_pickle('df_vital_opz.pkl')
df_hw_opz = pd.read_pickle('df_hw_opz.pkl')
df_death_opz = pd.read_pickle('pt_id/df_death_opz.pkl')
df_sex_opz = pd.read_pickle('df_sex_opz.pkl')
df_age_opz = pd.read_pickle('df_age_opz.pkl')
df_inj_opz = pd.read_pickle('pt_id/df_inj_opz.pkl')

In [ ]:
#read PT_ID (patinent ID)
list_PT_ID = list(df_inj_opz['PT_ID'])
list_PT_ID = list(set(list_PT_ID))
list_PT_ID = sorted(list_PT_ID)
print("#number of patients:",len(list_PT_ID))

#preprocessing vital data
def preprocess_df_vital_opz(df_vital_opz):
    clean_df_vital_opz = df_vital_opz.drop(['Time', '診療科', '呼吸数'], axis = 1)
    clean_df_vital_opz['体温'] = clean_df_vital_opz['体温'].astype(float)
    clean_df_vital_opz['脈拍'] = clean_df_vital_opz['脈拍'].astype(float)
    clean_df_vital_opz['血圧(最高)'] = clean_df_vital_opz['血圧(最高)'].astype(float)
    clean_df_vital_opz['血圧(最低)'] = clean_df_vital_opz['血圧(最低)'].astype(float)

    clean_df_vital_opz = clean_df_vital_opz.sort_values(by = 'Date')
    
    
    return clean_df_vital_opz

preprocess_df_vital_opz(df_vital_opz)

clean_df_vital_opz=preprocess_df_vital_opz(df_vital_opz)
clean_df_vital_opz=preprocess_df_vital_opz(df_vital_opz)
df_one_vital = clean_df_vital_opz[clean_df_vital_opz["PT_ID"]=="920076678"]
df_one_vital

#患者さん一人のデータでDateごとの中央値を出す
df_one_vital_taionn = df_one_vital.groupby('Date').agg({'体温': 'median'})
df_one_vital_myaku = df_one_vital.groupby('Date').agg({'脈拍': 'median'})
df_one_vital_blood_h = df_one_vital.groupby('Date').agg({'血圧(最高)': 'median'})
df_one_vital_blood_l = df_one_vital.groupby('Date').agg({'血圧(最低)': 'median'})
#4つをDateをkeyにconcat
df_one_vital_median = pd.concat([df_one_vital_taionn, df_one_vital_myaku, df_one_vital_blood_h, df_one_vital_blood_l], axis=1)
df_one_vital_median

###PT_IDのリストぶんforループ
#clean_df_vital_opzの正規化を行う
clean_df_vital_opz
#MinMax法
mm = preprocessing.MinMaxScaler()
ttt = clean_df_vital_opz
ttt = ttt.values[:, 2:6]
ttt_mm = mm.fit_transform(ttt)
list_columns = ['体温', '脈拍', '血圧(最高)', '血圧(最低)']
ttt_mm = pd.DataFrame(ttt_mm, columns = list_columns)
ttt_mm
ppp = clean_df_vital_opz.reset_index(drop=True)
ppp = ppp.drop(['体温', '脈拍', '血圧(最高)', '血圧(最低)'], axis = 1)
pt_mmm = pd.concat([ttt_mm, ppp], axis = 1)
clean_df_vital_opz = pt_mmm

all_data_vital=[]
for x in list_PT_ID:
    #患者さん一人ぶん
    df_x_vital = clean_df_vital_opz[clean_df_vital_opz["PT_ID"]==str(x)]
    #患者さん一人のデータでDateごとの中央値を出す
    df_vital_taionn = df_x_vital.groupby('Date').agg({'体温': 'median'})
    df_vital_myaku = df_x_vital.groupby('Date').agg({'脈拍': 'median'})
    df_vital_blood_h = df_x_vital.groupby('Date').agg({'血圧(最高)': 'median'})
    df_vital_blood_l = df_x_vital.groupby('Date').agg({'血圧(最低)': 'median'})

    #4つをDateをkeyにconcat
    df_vital_median = pd.concat([df_vital_taionn, df_vital_myaku, df_vital_blood_h, df_vital_blood_l], axis=1)
    #print(df_vital_median)
    
    all_data_vital.append(df_vital_median)

all_data_vital[1]

In [ ]:
#injきれいにしたい
def preprocess_df_inj_opz(df_inj_opz):
    #injきれいにしたい
    clean_df_inj_opz = df_inj_opz.drop(['入力日', '薬剤名', '薬剤コード', '診療科コード', '診療科', '入力時刻', '更新時刻','更新日','キャンセル日','キャンセル時刻','キャンセル種別','指示用量','指示単位','成分単位量','成分単位','手技名','用法内容','実施状況','入外区分'], axis = 1)

    clean_df_inj_opz['薬価基準収載医薬品コード'].value_counts()
    #このうち抗がん剤のみ抽出する
    clean_df_inj_opz_423 = clean_df_inj_opz[df_inj_opz['薬価基準収載医薬品コード'].str.contains('423', na = False)]
    clean_df_inj_opz_424 = clean_df_inj_opz[df_inj_opz['薬価基準収載医薬品コード'].str.contains('424', na = False)]
    clean_df_inj_opz_429 = clean_df_inj_opz[df_inj_opz['薬価基準収載医薬品コード'].str.contains('429', na = False)]
    clean_df_inj_opz = pd.concat([clean_df_inj_opz_423, clean_df_inj_opz_424, clean_df_inj_opz_429])
    
    ###########頭文字でないの9個まぎれていたので消したい
    ##'3999424G1025','6343424D3019','1242405A1038','1242401A1315','6343424D2012','1242406G1035','6363424D3035', '3999423A1125','3999423A1044'
    drop_index_inj3999424G1025 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '3999424G1025']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj3999424G1025)

    drop_index_inj6343424D3019 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '6343424D3019']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj6343424D3019)

    drop_index_inj1242405A1038 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '1242405A1038']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj1242405A1038)

    drop_index_inj1242401A1315 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '1242401A1315']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj1242401A1315)

    drop_index_inj6343424D2012 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '6343424D2012']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj6343424D2012)

    drop_index_inj1242406G1035 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '1242406G1035']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj1242406G1035)

    drop_index_inj6363424D3035 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '6363424D3035']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj6363424D3035)

    drop_index_inj3999423A1125 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '3999423A1125']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj3999423A1125)

    drop_index_inj3999423A1044 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '3999423A1044']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj3999423A1044)
    
    ###########
    clean_df_inj_opz['有＝１'] = 1
    
    

    return clean_df_inj_opz

clean_df_inj_opz=preprocess_df_inj_opz(df_inj_opz)
###PT_IDのリストぶんforループ
all_data_inj=[]
for x in list_PT_ID:
    #患者一人ぶん
    df_one_inj = clean_df_inj_opz[clean_df_inj_opz["PT_ID"]==str(x)]

    #dateをリストに変換
    #そして重複を削除
    #並び替え
    list_one_inj_date = list(set(df_one_inj['Date']))
    list_one_inj_date.sort()

    #list_date_injは仮　本当はlist_one_inj_date
    list_date_inj = list_one_inj_date #['2017-07-03','2017-06-05']
    list_one_inj = []
    for x in list_date_inj:
        #Date=xの時のデータ
        df_one_inj_x = df_one_inj[df_one_inj['Date'] == str(x)]
        df_one_inj_x = df_one_inj_x.drop(['PT_ID', 'Date'], axis = 1)
        df_one_inj_x = df_one_inj_x.set_index("薬価基準収載医薬品コード")
        ## 注射二回打っている？場合の処理
        grouped = df_one_inj_x.groupby(level=0)  
        df_one_inj_x = grouped.last() 

        #date×検査項目にする
        df_one_inj_x=df_one_inj_x.T
        df_one_inj_x = df_one_inj_x.rename(index = {'有＝１': str(x)})
        list_one_inj.append(df_one_inj_x)
    merged_df_one_inj=pd.concat(list_one_inj)
    all_data_inj.append(merged_df_one_inj)

In [ ]:
##薬価基準収載医薬品コードはじめの7桁が主成分なので最初の7桁でグループ分け？する
clean_df_inj_opz['薬価基準収載医薬品コード最初７桁'] = clean_df_inj_opz['薬価基準収載医薬品コード'].str[:7]
list_namecode = [clean_df_inj_opz['薬価基準収載医薬品コード最初７桁']]
for x in list_namecode:
    list_namecode_ = 'inj'+ x
 
    clean_df_inj_opz = clean_df_inj_opz.drop(['薬価基準収載医薬品コード最初７桁'], axis = 1)
    clean_df_inj_opz['薬価基準収載医薬品コード最初７桁'] = list_namecode_
clean_df_inj_opz

####PT_IDのリストぶんforループ
all_data_inj=[]
for x in list_PT_ID:
    #患者一人ぶん
    df_one_inj = clean_df_inj_opz[clean_df_inj_opz["PT_ID"]==str(x)]

    #dateをリストに変換
    #そして重複を削除
    #並び替え
    list_one_inj_date = list(set(df_one_inj['Date']))
    list_one_inj_date.sort()

    #list_date_injは仮　本当はlist_one_inj_date
    list_date_inj = list_one_inj_date #['2017-07-03','2017-06-05']
    list_one_inj = []
    for x in list_date_inj:
        #Date=xの時のデータ
        df_one_inj_x = df_one_inj[df_one_inj['Date'] == str(x)]
        df_one_inj_x = df_one_inj_x.drop(['PT_ID', 'Date', '薬価基準収載医薬品コード'], axis = 1)
        df_one_inj_x = df_one_inj_x.set_index("薬価基準収載医薬品コード最初７桁")
        ## 注射二回打っている？場合の処理
        grouped = df_one_inj_x.groupby(level=0)  
        df_one_inj_x = grouped.last() 

        #date×検査項目にする
        df_one_inj_x=df_one_inj_x.T
        df_one_inj_x = df_one_inj_x.rename(index = {'有＝１': str(x)})
        list_one_inj.append(df_one_inj_x)
    merged_df_one_inj=pd.concat(list_one_inj)
    all_data_inj.append(merged_df_one_inj)

all_data_inj[0]

In [ ]:
c_inj_codes = clean_df_inj_opz['薬価基準収載医薬品コード最初７桁'].value_counts()
c_inj_codes = pd.DataFrame(c_inj_codes)
#####injコード100回以上登場する項目のみ採用する
#inj_codesは使うinjの項目たち
inj_codes = c_inj_codes.index[c_inj_codes['薬価基準収載医薬品コード最初７桁'] > 100]
inj_codes = list(inj_codes)
inj_codes

df_inj_codes = pd.DataFrame(columns = inj_codes)
df_inj_codes
for x in range(447):
    all_data_inj[x] = pd.concat([df_inj_codes, all_data_inj[x]], join = 'inner')
all_data_inj[6]

In [ ]:
#labきれいにしたい

#aaa = df_lab_opz[df_lab_opz['検査項目'] == '測定機器']
#aaa['検査項目コード'].value_counts()
##検査項目が測定機器のうち検査項目コード内訳は
###0270100:
###4315000
###0259000
###4325000
###9329090
#検査項目が測定機器なのを全部消す
sokuteikiki_index = df_lab_opz.index[df_lab_opz['検査項目'] == '測定機器']
sokuteikiki = df_lab_opz.drop(sokuteikiki_index)
df_lab_opz = sokuteikiki

def preprocess_df_lab_opz(df_lab_opz):
    #labきれいにしたい
    clean_df_lab_opz =  df_lab_opz.drop(['Time', '検査材料', '検査項目', 'フリーコメント','Value'], axis = 1)
    clean_df_lab_opz = clean_df_lab_opz.sort_values(['PT_ID', 'Date', '検査項目コード'])

    return df_lab_opz


clean_df_lab_opz=preprocess_df_lab_opz(df_lab_opz)

### 並列化高速版
###
from multiprocessing import Pool

def process_one_lab(x):
    print("Processing: ",x)
    #患者一人ぶん
    df_one_lab = clean_df_lab_opz[clean_df_lab_opz["PT_ID"]==str(x)]

    #dateをリストに変換
    #そして重複を削除
    #並び替え
    list_one_lab_date = list(set(df_one_lab['Date']))
    list_one_lab_date.sort()

    #list_dateは仮　本当はlist_one_lab_date
    list_date = list_one_lab_date #['2009-08-20', '2009-10-07']
    list_one_lab = []
    for x in list_date:
        #Date=xの時のデータ
        df_one_x = df_one_lab[df_one_lab['Date']==x]
        df_one_x = df_one_x.drop(['PT_ID', 'Date','Time'], axis = 1)
        df_one_x = df_one_x.set_index("検査項目コード")
        ## 二回検査している？場合の処理
        grouped = df_one_x.groupby(level=0)  
        df_one_x = grouped.last()
        #df_one_x["異常値"]
        df_one_x = df_one_x[["異常値"]].T

        #date×検査項目にする
        df_one_x = df_one_x.rename(index = {'異常値': ''+str(x)+''})
        list_one_lab.append(df_one_x)
    merged_df_one_lab=pd.concat(list_one_lab)
    return merged_df_one_lab

clean_df_lab_opz=preprocess_df_lab_opz(df_lab_opz)
p = Pool(32) # 並列プロセス数を設定
all_data_lab = p.map(process_one_lab, list_PT_ID)

In [ ]:
all_data_lab

In [ ]:
##lab頻出１００で相関
#相関係数を算出してabs0.7より大きい項目は相関が高いということで除く
lab_head = df_lab_opz['検査項目コード'].value_counts()
df_lab_head = pd.DataFrame(lab_head)
df_lab_head = df_lab_head.head(200)
list_lab_head = df_lab_head.index
list_lab_head

lab100 = []
for x in range(447):
    #labdataからよくある検査項目コードhead100を抽出したい
    lablab_data = all_data_lab[x]
    lablab_data = lablab_data.filter(items = list_lab_head)
    #＋L:-1、＋H:1、-:0にする
    lablab_data = lablab_data.replace({'＋Ｈ':1, '−':0, '＋Ｌ':-1})
    
    lab100.append(lablab_data)

all_data_lab_merge = pd.concat(lab100)

#lab相関見たい
#自信なし
df_lab_corr = all_data_lab_merge.corr()
print(df_lab_corr)

fig, ax = plt.subplots(figsize=(12, 9)) 
sns.heatmap(df_lab_corr, square=True, vmax=1, vmin=-1, center=0)
plt.savefig('pt_id/lab100sokan.png')
##相関高い特徴量を除外したい
##相関係数0.7≦|r|≦1.0は強い相関があるとして片割れは除外
import math
threshold = 0.7 #しきい値
# 相関行列を求める
lab_corr = all_data_lab_merge.corr()
print(lab_corr)

while True:
    columns = lab_corr.columns 
    data_max = 0.0
    index_max  = None
    column_max = None
    #相関行列の中で最も相関の高い組を1つ取り出す。
    for index in columns:
        for column in columns:
            data = abs(lab_corr.loc[index, column])
            # 自身同士でなく、しきい値を超えているか。
            if index != column and not np.isnan(data) and data > threshold :
                #相関の最も高いものを記憶
                if data_max < data:
                    data_max = data
                    index_max = index 
                    column_max = column 

    if data_max == 0.0:
        # しきい値を超えるものがなければ終了
        break
    else:
        # しきい値を超えるものがある場合、column側の特徴を相関行列から除去(行、列共に)
        lab_corr.drop([column_max], axis=0, inplace=True)
        lab_corr.drop([column_max], axis=1, inplace=True)

lab_corr  

In [ ]:
#for i in list_lab_head_corr:
    #toptop = all_data_lab_merge[str(i)].value_counts()
    #print(toptop)

In [ ]:
#list_lab_head_corr

In [ ]:
#頻出１００labの相関0.7以上を除いた94のtop50出す
#異常多いtop50と普通top50全く同じだったから、普通のtop50でやってる
##94項目のリスト
list_lab_corr = lab_corr.columns.tolist()
#DKFに入れたいlabのデータ
#検査項目コード多い上位100個のリストを作りたい
#lab_head = df_lab_opz['検査項目コード'].value_counts()
#df_lab_head = pd.DataFrame(lab_head)
#df_lab_head_corr = pd.DataFrame(index = list_lab_corr)
#df_lab_head_corr = pd.concat([df_lab_head, df_lab_head_corr], axis = 0, join = 'inner')

#df_lab_head_corr = df_lab_head_corr.head(50)
#list_lab_head_corr = df_lab_head_corr.index
#list_lab_head_corr
#labデータ異常値が＋のもののみいったん出して見る
#異常値が多いものです
df_lab_opz_H = df_lab_opz[df_lab_opz['異常値'] == '＋Ｈ']
df_lab_opz_L = df_lab_opz[df_lab_opz['異常値'] == '＋Ｌ']
df_lab_opz_HL = pd.concat([df_lab_opz_H, df_lab_opz_L])
vc_df_lab_opz_HL = df_lab_opz_HL['検査項目コード'].value_counts()
vc_df_lab_opz_HL = pd.DataFrame(vc_df_lab_opz_HL)
vc_df_lab_opz_HL
df_lab_head_corr = pd.DataFrame(index = list_lab_corr)
df_lab_head_corr = pd.concat([vc_df_lab_opz_HL, df_lab_head_corr], axis = 0, join = 'inner')

df_lab_head_corr = df_lab_head_corr.head(50)
list_lab_head_corr = df_lab_head_corr.index
list_lab_head_corr
df_lab_head_corr



lab50_corr = []
for x in range(447):
    #labdataからよくある検査項目コードhead100を抽出したい
    lablab_data = all_data_lab[x]
    lablab_data = lablab_data.filter(items = list_lab_head_corr)
    #＋L:-1、＋H:1、-:0にする##MMscaler考慮
    lablab_data = lablab_data.replace({'＋Ｈ':1, '−':0.5, '＋Ｌ':0})
    list_columns = lablab_data.columns.tolist()
    for i in list_columns:
        df_bool = (lablab_data[str(i)] == 0)
        counts_df_bool = df_bool.sum()
        if counts_df_bool < 0:
            lablab_data[str(i)] = lablab_data[str(i)].replace(0.5, 0)
        else:
            lablab_data[str(i)] = lablab_data[str(i)]
    lab50_corr.append(lablab_data)
    
lab50_corr[1]
#list_lab_head_corr

In [ ]:
list_lab_head_corr

In [ ]:
#検査項目コードと検査項目の対応の表作ります
##検査項目コードと検査項目dfから抽出
k_df_lab_opz = df_lab_opz.loc[:,['検査項目コード','検査項目']]
k_df_lab_opz.drop_duplicates(subset = ['検査項目コード', '検査項目'], inplace = True)
list_k_df_lab_opz_one = []
for i in list_lab_head_corr:
    k_df_lab_opz_one = k_df_lab_opz[k_df_lab_opz["検査項目コード"] == str(i)]
    list_k_df_lab_opz_one.append(k_df_lab_opz_one)
k_merge_df_lab_opz = pd.concat(list_k_df_lab_opz_one)
############################
#用いている検査項目コードとの対応
#k_merge_df_lab_opz.to_csv("labcode_50.csv")

In [ ]:
k_merge_df_lab_opz

In [ ]:
#nameきれいにしたい
def preprocess_df_name_opz(df_name_opz):
    clean_df_name_opz =  df_name_opz.drop(['診療科コード', '診療科', '接頭語１', '接頭語２','接尾語１','接尾語２','ICD10コード(基礎疾患)', '病名', '疑い対象', '病名(＋修飾語)','転帰区分','転帰日'], axis = 1)
    #kラム名を検査コードと区別できるように変更
    list_namecode = [clean_df_name_opz['病名コード']]
    for x in list_namecode:
        list_namecode_ = '病名'+ x
 
    clean_df_name_opz = clean_df_name_opz.drop(['病名コード'], axis = 1)
    clean_df_name_opz['病名コード'] = list_namecode_
    
    clean_df_name_opz['有＝１'] = 1
    
    return clean_df_name_opz

clean_df_name_opz = preprocess_df_name_opz(df_name_opz)
#PT_IDぶんのforループ
all_data_name = []
for x in list_PT_ID:
    df_one_name = clean_df_name_opz[clean_df_name_opz["PT_ID"] == str(x)]
    
    #dateをリストに変換、重複削除、並び替え
    list_one_name_date = list(set(df_one_name['Date']))
    list_one_name_date.sort()
    
    list_date_name = list_one_name_date
    list_one_name = []
    
    for x in list_date_name:
        #Date = xの時のデータ
        df_one_name_x = df_one_name[df_one_name['Date'] == str(x)]
        df_one_name_x = df_one_name_x.drop(['PT_ID', 'Date'], axis = 1)
        df_one_name_x = df_one_name_x.set_index("病名コード")
        
        ##2回診断されている場合？の処理
        grouped = df_one_name_x.groupby(level = 0)
        df_one_name_x = grouped.last()

        
        #date×病名コードにする
        df_one_name_x = df_one_name_x.T
        df_one_name_x = df_one_name_x.rename(index = {'有＝１': str(x)})
        list_one_name.append(df_one_name_x)
    
    merged_df_one_name = pd.concat(list_one_name)
    all_data_name.append(merged_df_one_name)

In [ ]:
#病名の中から癌のみをピックアップ
df_name_opz['病名'].value_counts()
#icd=cが悪性新生物

df_name_cancer = df_name_opz[df_name_opz['ICD10コード(基礎疾患)'].str.startswith('C', na = False)]
print(df_name_cancer['ICD10コード(基礎疾患)'].value_counts())

#病名はICDの頭文字で分類する
df_name_opz
##ICDコード不明（302個）はいったん無視する～～～
#print(df_name_opz.isnull().sum())

#list_icd_x = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'N', 'M', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
#icd_name = []
#for x in list_icd_x:
    #icd_x = df_name_opz[df_name_opz['ICD10コード(基礎疾患)'].str.startswith(str(x), na = False)]
    #icd_x['ICD-X'] = str(x)
    #icd_name.append(icd_x)    
    
#icd_name = pd.concat([icd_name[0],icd_name[1],icd_name[2],icd_name[3],icd_name[4],icd_name[5],icd_name[6],icd_name[7],icd_name[8],icd_name[9],icd_name[10],icd_name[11],icd_name[12],icd_name[13],icd_name[14],icd_name[15],icd_name[16],icd_name[17],icd_name[18],icd_name[19],icd_name[20],icd_name[21],icd_name[22],icd_name[23],icd_name[24],icd_name[25]], axis = 0, join = 'outer')
#icd_name

icd_x = df_name_opz[df_name_opz['ICD10コード(基礎疾患)'].str.startswith('C', na = False)]
icd_x 

icd_x['ICD10コード(基礎疾患)最初3桁'] = icd_x['ICD10コード(基礎疾患)'].str[:3]

##C頭文字２個だと６３種と多いので（腫瘍リスト）作成
#あほみたいなことしているのは後で項目変えたいってなったと時用です
list_icd_x = pd.DataFrame(icd_x['ICD10コード(基礎疾患)最初3桁'].value_counts())
list_icd_x = list(list_icd_x.index)
list_icd_x = sorted(list_icd_x)
list_icd_x
list_c00_14 = ['C02','C03','C04','C05','C06','C07','C08','C10','C11','C13','C14']
list_c15_26 = ['C15','C16','C17','C18','C20','C22','C23','C24','C25','C26']
list_c30_39 = ['C30','C31','C32','C34','C38']
list_c40_41 = ['C41']
list_c43_44 = ['C43','C44']
list_c45_49 = ['C45','C48','C49']
list_c50 = ['C50']
list_c51_58 = ['C51','C52','C53','C54','C55','C56','C58']
list_c60_63 = ['C61']
list_c64_68 = ['C64','C65','C66','C67','C68']
list_c69_72 = ['C69','C71']
list_c73_75 = ['C73','C74']
list_c76_80 = ['C76','C77','C78','C79','C80']
list_c81_96 = ['C81','C83','C84','C85','C90','C91','C92','C95']

list_icd_c00_14 = []
for x in list_c00_14:
    list_icd_c00_14.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c00_14 = pd.concat(list_icd_c00_14)
df_icd_c00_14['がんグループ'] = 'c00_14'

list_icd_c15_26 = []
for x in list_c15_26:
    list_icd_c15_26.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c15_26 = pd.concat(list_icd_c15_26)
df_icd_c15_26['がんグループ'] = 'c15_26'

list_icd_c30_39 = []
for x in list_c30_39:
    list_icd_c30_39.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c30_39 = pd.concat(list_icd_c30_39)
df_icd_c30_39['がんグループ'] = 'c30_39'
 
list_icd_c40_41 = []
for x in list_c40_41:
    list_icd_c40_41.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c40_41 = pd.concat(list_icd_c40_41)
df_icd_c40_41['がんグループ'] = 'c40_41'

list_icd_c43_44 = []
for x in list_c43_44:
    list_icd_c43_44.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c43_44 = pd.concat(list_icd_c43_44)
df_icd_c43_44['がんグループ'] = 'c43_44'

list_icd_c45_49 = []
for x in list_c45_49:
    list_icd_c45_49.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c45_49 = pd.concat(list_icd_c45_49)
df_icd_c45_49['がんグループ'] = 'c45_49'

list_icd_c50 = []
for x in list_c50:
    list_icd_c50.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c50 = pd.concat(list_icd_c50)
df_icd_c50['がんグループ'] = 'c50'

list_icd_c51_58 = []
for x in list_c51_58:
    list_icd_c51_58.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c51_58 = pd.concat(list_icd_c51_58)
df_icd_c51_58['がんグループ'] = 'c51_58'

list_icd_c60_63 = []
for x in list_c60_63:
    list_icd_c60_63.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c60_63 = pd.concat(list_icd_c60_63)
df_icd_c60_63['がんグループ'] = 'c60_63'

list_icd_c64_68 = []
for x in list_c64_68:
    list_icd_c64_68.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c64_68 = pd.concat(list_icd_c64_68)
df_icd_c64_68['がんグループ'] = 'c64_68'

list_icd_c69_72 = []
for x in list_c69_72:
    list_icd_c69_72.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c69_72 = pd.concat(list_icd_c69_72)
df_icd_c69_72['がんグループ'] = 'c69_72'

list_icd_c73_75 = []
for x in list_c73_75:
    list_icd_c73_75.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c73_75 = pd.concat(list_icd_c73_75)
df_icd_c73_75['がんグループ'] = 'c73_75'

list_icd_c76_80 = []
for x in list_c76_80:
    list_icd_c76_80.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c76_80 = pd.concat(list_icd_c76_80)
df_icd_c76_80['がんグループ'] = 'c76_80'

list_icd_c81_96 = []
for x in list_c81_96:
    list_icd_c81_96.append(icd_x[icd_x['ICD10コード(基礎疾患)最初3桁'] == str(x)])
df_icd_c81_96 = pd.concat(list_icd_c81_96)
df_icd_c81_96['がんグループ'] = 'c81_96'


list_icd_xx = [df_icd_c00_14, df_icd_c15_26, df_icd_c30_39, df_icd_c40_41, df_icd_c43_44, df_icd_c45_49, df_icd_c50, df_icd_c51_58, df_icd_c60_63, df_icd_c64_68, df_icd_c69_72, df_icd_c73_75, df_icd_c76_80, df_icd_c81_96]
icd_x = pd.concat(list_icd_xx)

icd_name = icd_x

In [ ]:
#icd_nameきれいにしたい
def preprocess_icd_name_opz(icd_name):
    clean_icd_name_opz =  icd_name.drop(['診療科コード', 'ICD10コード(基礎疾患)最初3桁', '診療科', '病名コード', '接頭語１', '接頭語２','接尾語１','接尾語２','ICD10コード(基礎疾患)', '病名', '疑い対象', '病名(＋修飾語)','転帰区分','転帰日'], axis = 1)
    
    clean_icd_name_opz['有＝１'] = 1
    
    return clean_icd_name_opz

clean_icd_name_opz = preprocess_icd_name_opz(icd_name)

#都合上、PT_ID、Date、ICD-X　同じ行削除
clean_icd_name_opz.drop_duplicates(subset = ['PT_ID', 'Date', 'がんグループ'], inplace = True)

#PT_IDぶんのforループ
all_data_icd = []
for x in list_PT_ID:
    df_one_icd = clean_icd_name_opz[clean_icd_name_opz["PT_ID"] == str(x)]
    
    #dateをリストに変換、重複削除、並び替え
    list_one_icd_date = list(set(df_one_icd['Date']))
    list_one_icd_date.sort()
    list_date_icd = list_one_icd_date
    
    list_one_icd = []
    for x in list_date_icd:
        #Date = xの時のデータ
        df_one_icd_x = df_one_icd[df_one_icd['Date'] == str(x)]
        df_one_icd_x = df_one_icd_x.drop(['PT_ID', 'Date'], axis = 1)
        df_one_icd_x = df_one_icd_x.set_index("がんグループ")
    
        #date×ICD-Xにする
        df_one_icd_x = df_one_icd_x.T
        df_one_icd_x = df_one_icd_x.rename(index = {'有＝１': str(x)})
        list_one_icd.append(df_one_icd_x)
        #2回診断されている場合？の処理
        icd_grouped = df_one_icd_x.groupby(level = 0)
        df_one_icd_x = icd_grouped.last()

    merged_df_one_icd = pd.concat(list_one_icd)
    all_data_icd.append(merged_df_one_icd)

In [ ]:
all_data_icd_merge = pd.concat(all_data_icd)
all_data_icd_notnull = all_data_icd_merge.notnull().sum()
c_icd_codes = all_data_icd_notnull.sort_values()

c_icd_codes = pd.DataFrame(c_icd_codes)
c_icd_codes
#####injコード100回以上登場する項目のみ採用する
#inj_codesは使うinjの項目たち
icd_codes = c_icd_codes.index[c_icd_codes[0] > 100]
icd_codes = list(icd_codes)
icd_codes

In [ ]:
df_icd_codes = pd.DataFrame(columns = icd_codes)
df_icd_codes
for x in range(447):
    all_data_icd[x] = pd.concat([df_icd_codes, all_data_icd[x]], join = 'inner')
all_data_icd[6]

In [ ]:
list_c60_63_name = []
list_c64_68_name = []
list_c43_44_name = []
list_c00_14_name = []

list_c81_96_name = []
list_c15_26_name = []
list_c30_39_name = []
list_c76_80_name = []

for i in range(447):
    all_data_icd_one = all_data_icd[i]
    if 'c60_63' in all_data_icd_one.columns:
        list_c60_63_name.append(i)
    if 'c64_68' in all_data_icd_one.columns:
        list_c64_68_name.append(i)
    if 'c43_44' in all_data_icd_one.columns:
        list_c43_44_name.append(i)
    if 'c00_14' in all_data_icd_one.columns:
        list_c00_14_name.append(i)
        
    if 'c81_96' in all_data_icd_one.columns:
        list_c81_96_name.append(i)
    if 'c15_26' in all_data_icd_one.columns:
        list_c15_26_name.append(i)
    if 'c30_39' in all_data_icd_one.columns:
        list_c30_39_name.append(i)
    if 'c76_80' in all_data_icd_one.columns:
        list_c76_80_name.append(i)
    
len(list_c76_80_name)

In [ ]:
df_age_opz

In [ ]:
df_age_droupby = df_age_opz.groupby('PT_ID').agg({'estim_Age_groupnum': 'median'})
df_age_droupby = df_age_droupby.reset_index()
df_age_droupby.sort_values('PT_ID')

In [ ]:
df_age_droupby_1 = df_age_droupby[df_age_droupby['estim_Age_groupnum'] <2]
list_1 = df_age_droupby_1.index
df_age_droupby_2 = df_age_droupby.query('2<= estim_Age_groupnum <3')
list_2 = df_age_droupby_2.index
df_age_droupby_3 = df_age_droupby.query('3<= estim_Age_groupnum <4')
list_3 = df_age_droupby_3.index
df_age_droupby_4 = df_age_droupby.query('4<= estim_Age_groupnum <5')
list_4 = df_age_droupby_4.index

In [ ]:
len(list_4)

In [ ]:
#age正規化
df_age_opz
#MinMax法
mm = preprocessing.MinMaxScaler()

ttt = df_age_opz
ttt = ttt.values[:, 3:4]
ttt_mm = mm.fit_transform(ttt)
list_columns_age = ['estim_Age_groupnum']
ttt_mm = pd.DataFrame(ttt_mm, columns = list_columns_age)
ttt_mm
ppp = df_age_opz.reset_index(drop=True)
ppp = ppp.drop(list_columns_age, axis = 1)
pt_mmm = pd.concat([ttt_mm, ppp], axis = 1)
df_age_opz = pt_mmm
df_age_opz

In [ ]:
df_hw_opz

In [ ]:
#hw正規化
df_hw_opz
#MinMax法
mm = preprocessing.MinMaxScaler()

ttt = df_hw_opz
ttt = ttt.values[:, 2:4]
ttt_mm = mm.fit_transform(ttt)
list_columns_hw = ['身長', '体重']
ttt_mm = pd.DataFrame(ttt_mm, columns = list_columns_hw)
ttt_mm
ppp = df_hw_opz.reset_index(drop=True)
ppp = ppp.drop(list_columns_hw, axis = 1)
pt_mmm = pd.concat([ttt_mm, ppp], axis = 1)
df_hw_opz = pt_mmm
df_hw_opz

In [ ]:
#ageきれいにしたい
all_data_age = []
for x in list_PT_ID:
    df_one_age = df_age_opz[df_age_opz["PT_ID"] == str(x)]
    df_one_age_x = df_one_age.drop(['PT_ID', 'index'], axis = 1)
    
    all_data_age.append(df_one_age_x)
#hwきれいにしたい
all_data_hw = []
for x in list_PT_ID:
    df_one_hw = df_hw_opz[df_hw_opz["PT_ID"] == str(x)]
    df_one_hw_x = df_one_hw.drop(['PT_ID'], axis = 1)
    
    all_data_hw.append(df_one_hw_x)
#性別
df_sex_opz  = df_sex_opz.sort_values('PT_ID')
list_sex_opz = df_sex_opz['estim_bln_Male'].to_list()

columns作成

In [ ]:
###Dateを1回目二回目的な概念でとらえる？ためにタイムポイント（仮）を作る
#今回最もDate多いので752日だったので１～７５２のdf作成
##下位90パーで日数をカットすることにしました307日
lis_empty_timepoint = list(range(1, 307+1))
df_empty_timepoint = pd.DataFrame(columns = ['time_point_number'])
df_empty_timepoint['time_point_number'] = lis_empty_timepoint
#df_empty_timepoint

list_else = ['身長', '体重', 'estim_Age_groupnum', '体温', '脈拍', '血圧(最高)', '血圧(最低)', 'estim_bln_Male']
#カラムsから空のデータフレームを作成する
#hw,age.vital.sex.lab.inj,nameよりcolumns作成
#list_lab_head = list(list_lab_head)
#list_c_name_head = list(list_c_name_head)
list_lab_head_corr = list(list_lab_head_corr)
#薬価基準収載7桁のリスト作成
yyy = clean_df_inj_opz['薬価基準収載医薬品コード最初７桁'].value_counts()
yyy = pd.DataFrame(yyy)
list_inj7 = list(yyy.index)
#がん病名 ICD10コード(基礎疾患)最初3桁
zzz = clean_icd_name_opz['がんグループ'].value_counts()
zzz = pd.DataFrame(zzz)
list_icd = list(zzz.index)


list_inj7 = inj_codes
list_inj_7= inj_codes
list_icd = icd_codes

#list_empty_columns = list_else + list_lab_head + list_inj7 + list_icd
list_empty_columns = list_else + list_lab_head_corr + list_inj7 + list_icd
list_empty_columns

df_list_empty_columns = pd.DataFrame(columns = list_empty_columns)
df_list_empty_columns
##hw2,age1,vital4,sex1,lab100,inj34,name26の168カラムになりました

mergeする

In [ ]:
%%time
##カラム減らした完全バージョン
##all_data_と欠損値補完方法が異なる
#欠損は前の値で埋める
#ごみすぎる。。つら。
all_data = []
all_data_df = []
for x in range(447):
    pt_3 = pd.concat([lab50_corr[x], all_data_inj[x], all_data_icd[x]], axis = 1, join = 'outer')
    pt_1  = pd.merge(all_data_hw[x],all_data_age[x], on = 'Date', how = 'outer')
    pt_2 = pd.merge(pt_1,all_data_vital[x], on = 'Date', how = 'outer')
    pt_2 = pt_2.sort_values(['Date'])
    pt_3['Date'] = pt_3.index
    pt_3['Date'] = pd.to_datetime(pt_3['Date'])
    pt_3 = pt_3.sort_values(['Date'])

    pt_  = pd.merge(pt_2, pt_3, on = 'Date', how = 'outer')

    ##性別追加
    pt_["estim_bln_Male"] = list_sex_opz[x]
    pt_ = pt_.sort_values(['Date'])

    ##すべての項目追加
    pt_ = pd.concat([pt_, df_list_empty_columns], axis = 0, join = 'outer')

    ##タイムポイントを300日に設定しているので300より多い場合は早い日付けをカットする
    if len(pt_) > 307:
        pt_ = pt_.tail(307)
    else:
        pt_ = pt_
        
    #タイムポイント（仮）を振る
    xxx = len(pt_)
    lis_xxx = list(range(1, xxx+1))
    pt_["time_point_number"] = lis_xxx

    #タイムポイントを振った患者dfとタイムポイントのdfとタイムポイントをkeyに結合
    pt_ = pd.merge(pt_, df_empty_timepoint, on = 'time_point_number', how = 'outer')

    #timepointを消す
    pt_ = pt_.drop("time_point_number", axis = 1)
    

    #～～～処理～～～
    #age標準化
    #欠損ありなしに分けて、なしで標準化し結合
    #yyy = pt_['estim_Age_groupnum']
    #yyy_notnan = yyy[yyy.notnull()]
    #yyy_nan = yyy[yyy.isnull()]
    #yyy_notnan = (yyy_notnan - yyy_notnan.mean()) / yyy_notnan.std()
    #yyy_notnan = pd.DataFrame(yyy_notnan)
    #yyy_nan = pd.DataFrame(yyy_nan)
    #sc = StandardScaler()
    #yyy_notnan.loc[:, :] = sc.fit_transform(yyy_notnan)
    #yyy = pd.concat([yyy_notnan, yyy_nan])
    #インデックスでソート
    #yyy = yyy.sort_index()
    #pt_ = pt_.drop("estim_Age_groupnum", axis = 1)
    #pt_ = pd.merge(pt_, yyy, left_index=True, right_index=True)
    #age欠損は１つ前で埋める
    pt_['estim_Age_groupnum'] = pt_['estim_Age_groupnum'].fillna(method = 'ffill')
    #最初のデータより前はないから1番小さい値で埋めるで良きかな
    pt_age_min = pt_['estim_Age_groupnum'].min()
    pt_['estim_Age_groupnum'] = pt_['estim_Age_groupnum'].fillna(pt_age_min)

    
    #injの欠損値を０で埋める（欠損でない）
    for i in list_inj7:
        pt_[str(i)] = pt_[str(i)].fillna(0)
        #injを一応floatへ
        pt_[str(i)] = pt_[str(i)].astype('float')   
        
    #nameは1までを０、１からは全て１にする（欠損でない）
    for n in list_icd:
        pt_[str(n)] = pt_[str(n)].fillna(method = 'ffill')
        pt_[str(n)] = pt_[str(n)].fillna(0)
        #nameを一応floatへ
        pt_[str(n)] = pt_[str(n)].astype('float')
        #いったんタイムステップあるとこまででカットする
        #len_steps = len(pt_['Date'])
        #pt_steps = pt_.iloc[:len_steps+1, :]
        #標準化
        #yyy = pt_steps[str(n)]
        #yyy_notnan = yyy[yyy.notnull()]
        #yyy_nan = yyy[yyy.isnull()]
        ##yyy_notnan = (yyy_notnan - yyy_notnan.mean()) / yyy_notnan.std()
        #yyy_notnan = pd.DataFrame(yyy_notnan)
        #yyy_nan = pd.DataFrame(yyy_nan)
        #sc = StandardScaler()
        #yyy_notnan.loc[:, :] = sc.fit_transform(yyy_notnan)
        #yyy = pd.concat([yyy_notnan, yyy_nan])
        #インデックスでソート
        #yyy = yyy.sort_index()
        #pt_ = pt_.drop(str(n), axis = 1)
        #pt_ = pd.merge(pt_, yyy, left_index=True, right_index=True)
        #一応step外も埋めておく
        #pt_[str(n)] = pt_[str(n)].fillna(method = 'ffill')
        
        
        
    #身長は欠損でない
    pt_['体重'] = pt_['体重'].astype(float)
    pt_['身長'] = pt_['身長'].astype(float)
    #身長、体重、vital標準化
    #list_yyy = ['身長', '体重', '体温', '脈拍', '血圧(最高)', '血圧(最低)']
    #for i in list_yyy:
        #yyy = pt_[str(i)]
        #yyy_notnan = yyy[yyy.notnull()]
        #yyy_nan = yyy[yyy.isnull()]
        ##yyy_notnan = (yyy_notnan - yyy_notnan.mean()) / yyy_notnan.std()
        #yyy_notnan = pd.DataFrame(yyy_notnan)
        #yyy_nan = pd.DataFrame(yyy_nan)
        #sc = StandardScaler()
        #yyy_notnan.loc[:, :] = sc.fit_transform(yyy_notnan)
        #yyy = pd.concat([yyy_notnan, yyy_nan])
        #インデックスでソート
        #yyy = yyy.sort_index()
        #pt_ = pt_.drop(str(i), axis = 1)
        #pt_ = pd.merge(pt_, yyy, left_index=True, right_index=True)
        
    pt_['身長'] = pt_['身長'].fillna(method = 'ffill')    
    pt_['身長'] = pt_['身長'].fillna(pt_['身長'].mean()) 
    
    ##Dateの1個前のDateとの差Δtを求める
    #Δt求める
    pt_['Δt'] = (pt_['Date'].diff()).dt.days
    pt_['Δt'] = pt_['Δt'].astype('float')
    #Δt標準化する
    #yyy = pt_['Δt']
    #yyy_notnan = yyy[yyy.notnull()]
    #yyy_nan = yyy[yyy.isnull()]
    ##yyy_notnan = (yyy_notnan - yyy_notnan.mean()) / yyy_notnan.std()
    #yyy_notnan = pd.DataFrame(yyy_notnan)
    #yyy_nan = pd.DataFrame(yyy_nan)
    #sc = StandardScaler()
    #yyy_notnan.loc[:, :] = sc.fit_transform(yyy_notnan)
    #yyy = pd.concat([yyy_notnan, yyy_nan])
    #インデックスでソート
    #yyy = yyy.sort_index()
    #pt_ = pt_.drop("Δt", axis = 1)
    #pt_ = pd.merge(pt_, yyy, left_index=True, right_index=True)
    
    pt_ = pt_.drop(columns = 'Date')
    
    
    #labをfloatへ
    for j in list_lab_head_corr:
        pt_[str(j)] = pt_[str(j)].astype('float')
        #labを標準化する
        #yyy = pt_[str(j)]
        #yyy_notnan = yyy[yyy.notnull()]
        #yyy_nan = yyy[yyy.isnull()]
        ##yyy_notnan = (yyy_notnan - yyy_notnan.mean()) / yyy_notnan.std()
        #yyy_notnan = pd.DataFrame(yyy_notnan)
        #yyy_nan = pd.DataFrame(yyy_nan)
        #if yyy.isnull().all():
            #完全な欠損（1回もその検査を行っていない）は問題なくて検査しなかったのかなという判断で０埋めする。。。？
            #pt_[str(j)] = 0
        #else:
            #sc = StandardScaler()
            #yyy_notnan.loc[:, :] = sc.fit_transform(yyy_notnan)
            #yyy = pd.concat([yyy_notnan, yyy_nan])
            #インデックスでソート
            #yyy = yyy.sort_index()
            #pt_ = pt_.drop(str(j), axis = 1)
            #pt_ = pd.merge(pt_, yyy, left_index=True, right_index=True)
 
 
    
    ##～～～～欠損処理～～～～
    ##Date欠損どうしよう～～～とりあえず０で埋める
    pt_['Δt'] = pt_['Δt'].fillna(0)
    pt_ = pd.concat([df_list_empty_columns, pt_], axis = 0, join = 'outer')
    #正規化
    #MinMax法
    mm = preprocessing.MinMaxScaler()

    ttt = pt_
    ttt = ttt.values[:, 80:81]
    ttt_mm = mm.fit_transform(ttt)
    list_columns_t = ['Δt']
    ttt_mm = pd.DataFrame(ttt_mm, columns = list_columns_t)
    ttt_mm
    ppp = pt_.reset_index(drop=True)
    ppp = ppp.drop(list_columns_t, axis = 1)
    pt_mmm = pd.concat([ttt_mm, ppp], axis = 1)
    pt_ = pt_mmm
    
    
    
    
    #lab
    for j in list_lab_head_corr:
        #欠損値を１つ前の値で埋める
        #######↑NEW
        pt_[str(j)] = pt_[str(j)].fillna(method = 'ffill')
        #最初の値より前の欠損値は問題なしとして０で埋める
        ######↑NEW
        #完全な欠損（1回もその検査を行っていない）は問題なくて検査しなかったのかなという判断で０埋めする。。。？
        
        ###0.5埋め！！！
        pt_[str(j)] = pt_[str(j)].fillna(0.5)
    
    ##欠損値を１つ前の値で埋める
    pt_ = pt_.fillna(method = 'ffill') 
    #最初の欠損より前の値は平均で埋める
    #(言っても体重とバイタルくらいかな)
    pt_ = pt_.fillna(pt_.mean())
    
    ##標準化する（平均０標準偏差１になるようにする）
    #sc = StandardScaler()
    #pt_.loc[:, :] = sc.fit_transform(pt_) 
    pt_ = pd.concat([df_list_empty_columns, pt_], axis = 0, join = 'outer')
    
    pt_['Δt'] = pt_['Δt'].astype('float')
    pt_['estim_bln_Male'] = pt_['estim_bln_Male'].astype('float')
##numpy 3次元へ変換
    all_data.append(pt_.values)
    all_data_df.append(pt_) 

In [ ]:
%%time
#alldataとall_data_verでmaskは同じだが一応！！！
##カラム減らしたバージョン
##mask_verをつくりたい
###all_data_maskという名前で保存してdataに入れたい
##欠損を０、データあるところを１としたもの
all_data_mask = []
all_data_mask_df = []
for x in range(447):
    pt_3 = pd.concat([lab50_corr[x], all_data_inj[x], all_data_icd[x]], axis = 1, join = 'outer')
    pt_1  = pd.merge(all_data_hw[x],all_data_age[x], on = 'Date', how = 'outer')
    pt_2 = pd.merge(pt_1,all_data_vital[x], on = 'Date', how = 'outer')
    
    pt_3['Date'] = pt_3.index
    pt_3['Date'] = pd.to_datetime(pt_3['Date'])

    pt_  = pd.merge(pt_2, pt_3, on = 'Date', how = 'outer')

    ##性別追加
    pt_["estim_bln_Male"] = list_sex_opz[x]
    pt_ = pt_.sort_values(['Date'])

       ##すべての項目追加
    df_list_else = pd.DataFrame(columns = list_else)
    pt_ = pd.concat([pt_, df_list_else], axis = 0, join = 'outer')
    pt_ = pd.concat([pt_, df_list_empty_columns], axis = 0, join = 'outer')

    ##タイムポイントを300日に設定しているので300より多い場合は早い日付けをカットする
    if len(pt_) > 307:
        pt_ = pt_.tail(307)
    else:
        pt_ = pt_
        
    #タイムポイント（仮）を振る
    xxx = len(pt_)
    lis_xxx = list(range(1, xxx+1))
    pt_["time_point_number"] = lis_xxx
    #タイムポイントを振った患者dfとタイムポイントのdfとタイムポイントをkeyに結合
    pt_ = pd.merge(pt_, df_empty_timepoint, on = 'time_point_number', how = 'outer')
  
    #timepointを消す
    pt_ = pt_.drop("time_point_number", axis = 1)
    
    
    #～～～欠損値処理～～～
    #age欠損は１つ前で埋める
    pt_['estim_Age_groupnum'] = pt_['estim_Age_groupnum'].fillna(method = 'ffill')
    #最初のデータより前はないから1番小さい値で埋めるで良きかな
    pt_age_min = pt_['estim_Age_groupnum'].min()
    pt_['estim_Age_groupnum'] = pt_['estim_Age_groupnum'].fillna(pt_age_min)
    
    #injの欠損値を０で埋める。注射なしは０で欠損でない。
    for i in list_inj7:
        pt_[str(i)] = pt_[str(i)].fillna(0)
        #injを一応floatへ
        pt_[str(i)] = pt_[str(i)].astype('float')
    #nameのは1までを０、1からは全部１で埋める
    for n in list_icd:
        pt_[str(n)] = pt_[str(n)].fillna(method = 'ffill')
        pt_[str(n)] = pt_[str(n)].fillna(0)
    #nameを一応floatへ
        pt_[str(n)] = pt_[str(n)].astype('float')
    #labをfloatへ
    for j in list_lab_head_corr:
        pt_[str(j)] = pt_[str(j)].astype('float')
        #欠損値を平均で埋め、ない。
        #完全な欠損（1回もその検査を行っていない）は問題なくて検査しなかったのかなという判断で０埋め、ない。
    ##身長体重objectからfloatへ
    pt_['身長'] = pt_['身長'].astype(float)
    pt_['身長'] = pt_['身長'].fillna(pt_['身長'].mean())
    
    pt_['体重'] = pt_['体重'].astype(float)
    ##欠損値を平均で埋め、ない。
    
    ##Dateの1個前のDateとの差Δtを求める
    #Δt求める
    pt_['Δt'] = (pt_['Date'].diff()).dt.days
    pt_['Δt'] = pt_['Δt'].astype('float')
    ##Date欠損どうしよう～～～とりあえず０で埋め、ない。
    pt_ = pt_.drop(columns = 'Date')

    ##欠損以外を１で埋める
        #pt_[pt_.notnull()] = 1
    ##欠損を０で埋める
        #pt_.fillna(0, inplace = True) 
        
    ##こっちのほうが良さげ
    pt_isnull = pt_.isnull()
    pt_mask = pt_isnull.replace({True : 0, False : 1})
    
    pt_mask = pd.concat([df_list_empty_columns, pt_mask], axis = 0, join = 'outer')
    for j in list_empty_columns:
        pt_mask[str(i)] = pt_mask[str(i)].astype('float')
    pt_mask['Δt'] = pt_mask['Δt'].astype('float')
    pt_mask['estim_bln_Male'] = pt_mask['estim_bln_Male'].astype('float')
    pt_mask = pt_mask.astype('float')
##numpy 3次元へ変換
    all_data_mask.append(pt_mask.values)
    all_data_mask_df.append(pt_mask)

#欠損率を見たい
all_data_mask_df[0]

#有効なタイムステップでカット
all_data_mask_df_ = []
for i in range(447):
    x_dim2_one = all_data_mask_df[i]
    x_dim2_one = pd.DataFrame(x_dim2_one)
    all_data_steps_one = all_data_steps[i]
    x_dim2_one = x_dim2_one.head(all_data_steps_one)
    x_dim2_one['pt_no'] = i
    all_data_mask_df_.append(x_dim2_one)
all_data_mask_df_[0]
all_data_mask_df_merge = pd.concat(all_data_mask_df_)
all_data_mask_df_merge

all_data_mask_df_merge['time_steps'] = all_data_mask_df_merge.groupby('pt_no').cumcount()

all_data_mask_df_merge = pd.DataFrame(all_data_mask_df_merge).set_index(['pt_no', 'time_steps'])


#not_injname_kakuninn_mask_merge = pd.DataFrame(not_injname_kakuninn_mask_merge)
all_data_mask_df_merge

#項目ごとの欠損率
#項目ごとの欠損率
all_mask_nan = (all_data_mask_df_merge == 0).sum()#欠損あり数出してる
all_mask_nan = pd.DataFrame(all_mask_nan)
all_mask_nan = all_mask_nan.T
all_mask_nan_rate = all_mask_nan *100/73998##～欠損率です～
all_mask_nan

#全体の欠損率
all_mask_nan_all = all_mask_nan.sum(axis = 1)
all_mask_nan_allrate = all_mask_nan_all*100/(81*73998)

all_mask_nan_allrate###53.898734%

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 108)
all_data_df[0]

In [ ]:
all_data_df[0].info()

In [ ]:
#stepsも読み込み
import numpy
all_data_steps = numpy.load('pt_id/data/all_data_steps.npy')
all_data_steps[0]

In [ ]:
##カラム減らしたバージョン
#maskはall_dataと一緒ですたぶん

データセット分けたり

In [ ]:
#injとそれ以外のdf分けて作る
all_data_inj_df = []
inj_kakuninn=[]
all_data_not_inj = []
not_inj_kakuninn = []
for x in range(447):
    all_data_inj_one = all_data_df[x].filter(items=list_inj7, axis='columns')
    all_data_inj_df.append(all_data_inj_one.values)
    inj_kakuninn.append(all_data_inj_one)
    list_t = ['Δt']
    list_not_inj = list_else + list_lab_head_corr + list_icd + list_t
    all_data_not_inj_one = all_data_df[x].filter(items=list_not_inj, axis='columns')
    all_data_not_inj.append(all_data_not_inj_one.values)
    not_inj_kakuninn.append(all_data_not_inj_one)
print(all_data_inj_df[0].shape)
print(all_data_not_inj[0].shape)

In [ ]:
#injとそれ以外のdf分けて作る
#maskのも作る
#injnameとそれ以外のdf分けて作る
#maskのも作る
all_data_inj_mask_df = []
inj_kakuninn_mask = []
all_data_not_inj_mask = []
not_inj_kakuninn_mask = []
for i in range(447):
    all_data_inj_one = all_data_mask_df[i].filter(items = list_inj7, axis = 'columns')
    all_data_inj_mask_df.append(all_data_inj_one.values)
    inj_kakuninn_mask.append(all_data_inj_one)
    all_data_not_inj_one = all_data_mask_df[i].filter(items = list_not_inj, axis = 'columns')
    all_data_not_inj_mask.append(all_data_not_inj_one.values)
    not_inj_kakuninn_mask.append(all_data_not_inj_one)
#all_dataと同じ

In [ ]:
#欠損率を見たい
not_inj_kakuninn_mask[0]

#有効なタイムステップでカット
not_inj_kakuninn_mask_ = []
for i in range(447):
    x_dim2_one = not_inj_kakuninn_mask[i]
    x_dim2_one = pd.DataFrame(x_dim2_one)
    all_data_steps_one = all_data_steps[i]
    x_dim2_one = x_dim2_one.head(all_data_steps_one)
    x_dim2_one['pt_no'] = i
    not_inj_kakuninn_mask_.append(x_dim2_one)
not_inj_kakuninn_mask_[0]
not_inj_kakuninn_mask_merge = pd.concat(not_inj_kakuninn_mask_)
not_inj_kakuninn_mask_merge

not_inj_kakuninn_mask_merge['time_steps'] = not_inj_kakuninn_mask_merge.groupby('pt_no').cumcount()

not_inj_kakuninn_mask_merge = pd.DataFrame(not_inj_kakuninn_mask_merge).set_index(['pt_no', 'time_steps'])


#not_injname_kakuninn_mask_merge = pd.DataFrame(not_injname_kakuninn_mask_merge)
not_inj_kakuninn_mask_merge

In [ ]:
#項目ごとの欠損率
not_inj_nan = (not_inj_kakuninn_mask_merge == 0).sum()#欠損あり数出してる
not_inj_nan = pd.DataFrame(not_inj_nan)
not_inj_nan = not_inj_nan.T
not_inj_nan_rate = not_inj_nan *100/73998##～欠損率です～
not_inj_nan

#全体の欠損率
not_inj_nan_all = not_inj_nan.sum(axis = 1)
not_inj_nan_allrate = not_inj_nan_all*100/(67*73998)

not_inj_nan_allrate###53.898734%
not_inj_nan

In [ ]:
#timestepも同じ

In [ ]:
#injnameとそれ以外のdf分けて作る
all_data_injname_df = []
injname_kakuninn = []
all_data_not_injname = []
not_injname_kakuninn = []
list_injname = list_inj7 + list_icd
list_not_injname = list_else + list_lab_head_corr + list_t
for i in range(447):
    all_data_injname_one = all_data_df[i].filter(items = list_injname, axis = 'columns')
    all_data_injname_df.append(all_data_injname_one.values)
    injname_kakuninn.append(all_data_injname_one)
    all_data_not_injname_one = all_data_df[i].filter(items = list_not_injname, axis = 'columns')
    all_data_not_injname.append(all_data_not_injname_one.values)
    not_injname_kakuninn.append(all_data_not_injname_one)

In [ ]:
#injnameとそれ以外のdf分けて作る
#maskのも作る
all_data_injname_mask_df = []
injname_kakuninn_mask = []
all_data_not_injname_mask = []
not_injname_kakuninn_mask = []
for i in range(447):
    all_data_injname_one = all_data_mask_df[i].filter(items = list_injname, axis = 'columns')
    all_data_injname_mask_df.append(all_data_injname_one.values)
    injname_kakuninn_mask.append(all_data_injname_one)
    all_data_not_injname_one = all_data_mask_df[i].filter(items = list_not_injname, axis = 'columns')
    all_data_not_injname_mask.append(all_data_not_injname_one.values)
    not_injname_kakuninn_mask.append(all_data_not_injname_one)

In [ ]:
not_injname_kakuninn

In [ ]:
#欠損率を見たい
not_injname_kakuninn_mask[0]

#有効なタイムステップでカット
not_injname_kakuninn_mask_ = []
for i in range(447):
    x_dim2_one = not_injname_kakuninn_mask[i]
    x_dim2_one = pd.DataFrame(x_dim2_one)
    all_data_steps_one = all_data_steps[i]
    x_dim2_one = x_dim2_one.head(all_data_steps_one)
    x_dim2_one['pt_no'] = i
    not_injname_kakuninn_mask_.append(x_dim2_one)
not_injname_kakuninn_mask_[0]
not_injname_kakuninn_mask_merge = pd.concat(not_injname_kakuninn_mask_)
not_injname_kakuninn_mask_merge

not_injname_kakuninn_mask_merge['time_steps'] = not_injname_kakuninn_mask_merge.groupby('pt_no').cumcount()

not_injname_kakuninn_mask_merge = pd.DataFrame(not_injname_kakuninn_mask_merge).set_index(['pt_no', 'time_steps'])


#not_injname_kakuninn_mask_merge = pd.DataFrame(not_injname_kakuninn_mask_merge)
not_injname_kakuninn_mask_merge

In [ ]:
list_notinjname_columns = not_injname_kakuninn[0].columns

In [ ]:
print(list_notinjname_columns)

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 60)
not_injname_kakuninn[0]

In [ ]:
#項目ごとの欠損率
not_injname_nan = (not_injname_kakuninn_mask_merge == 0).sum()#欠損あり数出してる
not_injname_nan = pd.DataFrame(not_injname_nan)
not_injname_nan = not_injname_nan.T
not_injname_nan_rate = not_injname_nan *100/73998##～欠損率です～
not_injname_nan

#全体の欠損率
not_injname_nan_all = not_injname_nan.sum(axis = 1)
not_injname_nan_allrate = not_injname_nan_all*100/(59*73998)

not_injname_nan_allrate###61.207037%

In [ ]:
#timestepは同じ

In [ ]:
#deathもきれいにしたい
df_PT_ID = pd.DataFrame()
df_PT_ID['PT_ID'] = list_PT_ID
all_data_death = pd.merge(df_PT_ID, df_death_opz, on = 'PT_ID', how = 'outer')
print(all_data_death)
#死亡の有無のところ、NaNに０、＋に１を入れる
death_isnull = all_data_death.isnull()
all_data_death = death_isnull.replace({True : 0, False : 1})
all_data_death['PT_no'] = all_data_death.index
all_data_death.head(10)

In [ ]:
death_pt = all_data_death[all_data_death['死亡の有無'] == 1]
death_pt = list(death_pt['PT_no'])
not_death_pt = all_data_death[all_data_death['死亡の有無'] == 0]
not_death_pt = list(not_death_pt['PT_no'])

In [ ]:
d_notinjname_ver = []
s_notinjname_ver = []
d_notinjname_ver_mask = []
s_notinjname_ver_mask = []

d_notinj_ver = []
s_notinj_ver = []
d_notinj_ver_mask = []
s_notinj_ver_mask = []

d_ver = []
s_ver = []
d_ver_mask = []
s_ver_mask = []
d_ver_timestep = []
s_ver_timestep = []
for i in death_pt:
    notinjname_ver_one = not_injname_kakuninn[i]
    notinjname_ver_mask_one = not_injname_kakuninn_mask[i]
    d_notinjname_ver.append(notinjname_ver_one.values)
    d_notinjname_ver_mask.append(notinjname_ver_mask_one.values)
    
    d_notinj_ver_one = not_inj_kakuninn[i]
    d_notinj_ver_mask_one = not_inj_kakuninn_mask[i]
    d_notinj_ver.append(d_notinj_ver_one.values)
    d_notinj_ver_mask.append(d_notinj_ver_mask_one.values)
    
    d_ver_one = all_data_df[i]
    d_ver_mask_one = all_data_mask_df[i]
    d_ver_timestep_one = all_data_steps[i]
    d_ver.append(d_ver_one.values)
    d_ver_mask.append(d_ver_mask_one.values)
    d_ver_timestep.append(d_ver_timestep_one)
for i in not_death_pt:
    notinjname_ver_one = not_injname_kakuninn[i]
    notinjname_ver_mask_one = not_injname_kakuninn_mask[i]
    s_notinjname_ver.append(notinjname_ver_one.values)
    s_notinjname_ver_mask.append(notinjname_ver_mask_one.values)
    
    s_notinj_ver_one = not_inj_kakuninn[i]
    s_notinj_ver_mask_one = not_inj_kakuninn_mask[i]
    s_notinj_ver.append(s_notinj_ver_one.values)
    s_notinj_ver_mask.append(s_notinj_ver_mask_one.values)
    
    s_ver_one = all_data_df[i]
    s_ver_mask_one = all_data_mask_df[i]
    s_ver_timestep_one = all_data_steps[i]
    s_ver.append(s_ver_one.values)
    s_ver_mask.append(s_ver_mask_one.values)
    s_ver_timestep.append(s_ver_timestep_one)

保存等々

In [ ]:
#保存
#all_data_ver.npy
#all_inj_ver all_not_inj_ver

#リストからなんぱいへ変換
all_data = np.array(all_data)
all_data_inj_df = np.array(all_data_inj_df)
all_data_not_inj = np.array(all_data_not_inj)

all_data_injname_df = np.array(all_data_injname_df)
all_data_not_injname = np.array(all_data_not_injname)
all_data_injname_mask_df = np.array(all_data_injname_mask_df)
all_data_not_injname_mask = np.array(all_data_not_injname_mask)

#保存
np.save('pt_id/all_data_ver', all_data)
np.save('pt_id/all_inj_ver', all_data_inj_df)
np.save('pt_id/all_not_inj_ver', all_data_not_inj)

np.save('pt_id/all_injname_ver', all_data_injname_df)
np.save('pt_id/data/1215/all_not_injname_ver', all_data_not_injname)
np.save('pt_id/all_injname_ver_mask', all_data_injname_mask_df)
np.save('pt_id/data/1215/all_not_injname_ver_mask', all_data_not_injname_mask)


In [ ]:
#リストからなんぱいへ変換
d_notinjname_ver = np.array(d_notinjname_ver)
s_notinjname_ver = np.array(s_notinjname_ver)
d_notinjname_ver_mask = np.array(d_notinjname_ver_mask)
s_notinjname_ver_mask = np.array(s_notinjname_ver_mask)

d_notinj_ver = np.array(d_notinj_ver)
s_notinj_ver = np.array(s_notinj_ver)
d_notinj_ver_mask = np.array(d_notinj_ver_mask)
s_notinj_ver_mask = np.array(s_notinj_ver_mask)

d_ver = np.array(d_ver)
s_ver = np.array(s_ver)
d_ver_mask = np.array(d_ver_mask)
s_ver_mask = np.array(s_ver_mask)
d_ver_timestep = np.array(d_ver_timestep)
s_ver_timestep = np.array(s_ver_timestep)

#保存
np.save('pt_id/data/1215/d_notinjname_ver', d_notinjname_ver)
np.save('pt_id/data/1215/s_notinjname_ver', s_notinjname_ver)
np.save('pt_id/data/1215/d_notinjname_ver_mask', d_notinjname_ver_mask)
np.save('pt_id/data/1215/s_notinjname_ver_mask', s_notinjname_ver_mask)
np.save('pt_id/data/1215/d_notinj_ver', d_notinj_ver)
np.save('pt_id/data/1215/s_notinj_ver', s_notinj_ver)
np.save('pt_id/data/1215/d_notinj_ver_mask', d_notinj_ver_mask)
np.save('pt_id/data/1215/s_notinj_ver_mask', s_notinj_ver_mask)
np.save('pt_id/data/1215/d_ver', d_ver)
np.save('pt_id/data/1215/s_ver', s_ver)
np.save('pt_id/data/1215/d_ver_mask', d_ver_mask)
np.save('pt_id/data/1215/s_ver_mask', s_ver_mask)
np.save('pt_id/data/1215/d_ver_timestep',d_ver_timestep)
np.save('pt_id/data/1215/s_ver_timestep', s_ver_timestep)

テストjblよみこみ

In [ ]:
#stepsも読み込み
import numpy
import joblib
all_data_steps = numpy.load('pt_id/data/all_data_steps.npy')
all_data_steps[0]

#dim
#,lr=0.005
#ttt
#の潜在状態空間の点（ヒートマップのデータ）を読み込み
#["z_params"][0]=>平均：データ数　x タイムステップ x 状態空間次元
obj_dim2=joblib.load("pt_id/model/result_save_ds/notinjname_dim16/result_notinjname_dim16_lr01_FFF/test.jbl")
x_dim2 = obj_dim2["z_params"][0]

df_x_dim2 = []
for i in range(447):
    x_dim2_one = x_dim2[i]
    x_dim2_one = pd.DataFrame(x_dim2_one)
    all_data_steps_one = all_data_steps[i]
    x_dim2_one = x_dim2_one.head(all_data_steps_one)
    x_dim2_one['pt_no'] = i
    df_x_dim2.append(x_dim2_one)
print(df_x_dim2[0])

concat_df_x_dim2 = pd.concat(df_x_dim2)
concat_df_x_dim2 = concat_df_x_dim2.reset_index(drop=True)

list_pt_no = concat_df_x_dim2['pt_no']
df_pt_no = pd.DataFrame(list_pt_no)

#保存
df_pt_no.to_csv('pt_id/df_pt_no', index = False)

concat_df_x_dim2 = concat_df_x_dim2.drop(['pt_no'], axis = 1)


concat_df_x_dim2

#主成分分析で2次元にしてみる
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

pca.fit(concat_df_x_dim2)
pca_x_dim2 = pca.transform(concat_df_x_dim2)

pca_x_dim2 = pd.DataFrame(pca_x_dim2)

#解体作業
pca_x_dim2 = pd.concat([pca_x_dim2, df_pt_no], axis = 1, join = 'inner')
pca_x_dim2
pca_dim2 = []
for x in range(447):
    pca_i_dim2 = pca_x_dim2[pca_x_dim2['pt_no'] == x]
    pca_i_dim2 = pca_i_dim2.drop(['pt_no'], axis = 1)
    pca_dim2.append(pca_i_dim2)
pca_dim2[0]

In [ ]:
concat_df_x_dim2

In [ ]:
concat_df_x_dim2.to_csv('concat_df_pca_dkf16pca.csv')

In [ ]:
pca_dim2[0]

In [ ]:
#PCAでなくTSNEで次元削減してみる
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, perplexity = 30, random_state = 0)#perplexityの設定が大事らしい
vecs_list = tsne.fit_transform(concat_df_x_dim2)
vecs_list = pd.DataFrame(vecs_list)
vecs_list
#解体作業
tsne_x_dim2 = pd.concat([vecs_list, df_pt_no], axis = 1, join = 'inner')
tsne_x_dim2
tsne_dim2 = []
for x in range(447):
    tsne_i_dim2 = tsne_x_dim2[tsne_x_dim2['pt_no'] == x]
    tsne_i_dim2 = tsne_i_dim2.drop(['pt_no'], axis = 1)
    tsne_dim2.append(tsne_i_dim2)
tsne_dim2[0]

In [ ]:
vecs_list.to_csv('concat_df_dkf16tsne2.csv')

In [ ]:
plt.plot(tsne_dim2[1][0])
plt.plot(tsne_dim2[1][1])

クラスタリングの解析

In [ ]:
all_data_df_merge

In [ ]:
all_data_df_merge.to_csv('all_data_df_merge.csv')

In [ ]:
tsne_dim2_merge_cluster = list(tsne_dim2_merge['clusterABC'])
tsne_dim2_merge_cluster
all_data_df_merge['clusterABC'] = tsne_dim2_merge_cluster
all_data_df_merge

In [ ]:
all_data_clu_one = []
for x in range(447):
    all_data_df_merge_one = all_data_df_merge.query('pt_no == @x')
    all_data_clu_one.append(all_data_df_merge_one)
all_data_clu_one[0]

In [ ]:
#遷移定量化
clu_replace = []
for i in range(447):
    clu_replace_one = all_data_clu_one[i]
    clu_replace_one = clu_replace_one['clusterABC']
    clu_replace_one = clu_replace_one.replace({'clusterD':0, 'clusterC':0, 'clusterB':5, 'clusterA':3, 'cluster-1':'nan'})
    clu_replace_one = clu_replace_one.fillna(method = 'ffill')
    clu_replace_one = clu_replace_one.astype('float')
    clu_replace_one = clu_replace_one.diff()
    
    clu_replace_one = clu_replace_one.value_counts()
    clu_replace_one = pd.DataFrame(clu_replace_one)
    clu_replace_one = clu_replace_one.rename(columns = {'clusterABC': i})
    clu_replace.append(clu_replace_one)
    
clu_replace[0]

In [ ]:
death_pt

In [ ]:
clu_replace_all = pd.concat(clu_replace,axis = 1)
clu_replace_all = clu_replace_all.T
clu_replace_all.isnull().sum()

In [ ]:
all_data_cluA = all_data_df_merge[all_data_df_merge['clusterABC'] == 'clusterA'] 
all_data_cluB = all_data_df_merge[all_data_df_merge['clusterABC'] == 'clusterB']
all_data_cluC = all_data_df_merge[all_data_df_merge['clusterABC'] == 'clusterC'] 

In [ ]:
all_data_cluA

In [ ]:
#基本統計量出してみるぞい
#all_data_cluA、all_data_cluB、all_data_cluC
avgs_A = all_data_cluA.describe()
avgs_B = all_data_cluB.describe()
avgs_C = all_data_cluC.describe()
avgs_A#死亡左

In [ ]:
avgs_B#しぼう右

In [ ]:
avgs_C#何もないとこのこ

In [ ]:

a_columns = all_data_cluA.columns
for i in a_columns:
    #plt.boxplot(all_data_cluA[str(i)], vert=False)
    df_a = all_data_cluA[str(i)]    
    df_b = all_data_cluB[str(i)]
    df_c = all_data_cluC[str(i)]
    data = (df_a, df_b, df_c)
    figq, ax1 = plt.subplots()
    ax1.set_xticklabels(['A', 'B', 'C'])
    ax1.set_title(str(i))
    ax1.boxplot(data, showmeans=True)
    plt.show() 

In [ ]:
all_AAA = pd.concat([all_data_cluA['0201000'], all_data_cluA['0203000'], all_data_cluA['0202000'], all_data_cluA['0421000'], all_data_cluA['0206010'], all_data_cluA['0206040']],axis =1)
all_BBB = pd.concat([all_data_cluB['0201000'], all_data_cluB['0203000'], all_data_cluB['0202000'], all_data_cluB['0421000'], all_data_cluB['0206010'], all_data_cluB['0206040']],axis =1)
all_CCC = pd.concat([all_data_cluC['0201000'], all_data_cluC['0203000'], all_data_cluC['0202000'], all_data_cluC['0421000'], all_data_cluC['0206010'], all_data_cluC['0206040']],axis =1)

In [ ]:
all_data_cluA

In [ ]:
all_AAA_sum_05 = (all_AAA == 0.5).sum()
all_AAA_sum_0 = (all_AAA == 0).sum()
all_AAA_sum_1 = (all_AAA == 1).sum()

all_AAA_sum_0 = pd.DataFrame({'+L': all_AAA_sum_0})
all_AAA_sum_05 = pd.DataFrame({'-' : all_AAA_sum_05})
all_AAA_sum_1 = pd.DataFrame({'+H': all_AAA_sum_1})


all_AAA_sum = pd.concat([all_AAA_sum_05, all_AAA_sum_1, all_AAA_sum_0], axis = 1)
all_AAA_sum = all_AAA_sum.T

In [ ]:
all_BBB_sum_05 = (all_BBB == 0.5).sum()
all_BBB_sum_0 = (all_BBB == 0).sum()
all_BBB_sum_1 = (all_BBB == 1).sum()

all_BBB_sum_0 = pd.DataFrame({'+L': all_BBB_sum_0})
all_BBB_sum_05 = pd.DataFrame({'-' : all_BBB_sum_05})
all_BBB_sum_1 = pd.DataFrame({'+H': all_BBB_sum_1})


all_BBB_sum = pd.concat([all_BBB_sum_05, all_BBB_sum_1, all_BBB_sum_0], axis = 1)
all_BBB_sum = all_BBB_sum.T

In [ ]:
all_CCC_sum_05 = (all_CCC == 0.5).sum()
all_CCC_sum_0 = (all_CCC == 0).sum()
all_CCC_sum_1 = (all_CCC == 1).sum()

all_CCC_sum_0 = pd.DataFrame({'+L': all_CCC_sum_0})
all_CCC_sum_05 = pd.DataFrame({'-' : all_CCC_sum_05})
all_CCC_sum_1 = pd.DataFrame({'+H': all_CCC_sum_1})


all_CCC_sum = pd.concat([all_CCC_sum_05, all_CCC_sum_1, all_CCC_sum_0], axis = 1)
all_CCC_sum = all_CCC_sum.T
AAA_columns = all_CCC_sum.columns
AAA_columns

In [ ]:
all_CCC_sum = all_CCC_sum.reindex(index = ['+L', '-', '+H'])
all_AAA_sum = all_AAA_sum.reindex(index = ['+L', '-', '+H'])
all_BBB_sum = all_BBB_sum.reindex(index = ['+L', '-', '+H'])

In [ ]:
a_columns = all_data_cluA.columns
for x in AAA_columns:
    #plt.boxplot(all_data_cluA[str(i)], vert=False)
    df_a = pd.DataFrame({'A':all_AAA_sum[str(x)]})    
    df_b = pd.DataFrame({'B':all_BBB_sum[str(x)]})
    df_c = pd.DataFrame({'C':all_CCC_sum[str(x)]})
    dataset = pd.concat([df_a, df_b, df_c], axis = 1)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    for i in range(len(dataset)):
        ax.bar(dataset.columns, dataset.iloc[i], bottom=dataset.iloc[:i].sum())
    for j in range(len(dataset.columns)):
        plt.text(x=j, y=dataset.iloc[:i, j].sum() + (dataset.iloc[i, j] / 2), s=dataset.iloc[i, j], ha='center', va='bottom', fontsize = 20)
    #ax.set(xlabel='クラスター', ylabel='counts')
    ax.legend(dataset.index)
    plt.show()


In [ ]:
#colors = plt.get_cmap('Accent')(np.linspace(0, 1, 3))
colors = ('cornflowerblue', 'silver', 'magenta')
for x in AAA_columns:
    #plt.boxplot(all_data_cluA[str(i)], vert=False)
    df_a = pd.DataFrame({'A':all_AAA_sum[str(x)]})    
    df_b = pd.DataFrame({'B':all_BBB_sum[str(x)]})
    df_c = pd.DataFrame({'C':all_CCC_sum[str(x)]})
    dataset = pd.concat([df_a, df_b, df_c], axis = 1)
    
    
    plot_dataset = pd.DataFrame(index = dataset.index)
    for col in dataset.columns:
        plot_dataset[col] = round(100 * dataset[col] / dataset[col].sum(), 1)

    fig, ax = plt.subplots(figsize=(15, 10))
    for i in range(len(plot_dataset)):
        ax.bar(plot_dataset.columns, plot_dataset.iloc[i], bottom=plot_dataset.iloc[:i].sum(), color = colors[i])
        for j in range(len(plot_dataset.columns)):
            plt.text(x=j, y=plot_dataset.iloc[:i, j].sum()+(plot_dataset.iloc[i, j]/2), s=f'{plot_dataset.iloc[i, j]}%', ha='center', va='bottom', fontsize = 35 )
    #ax.set(xlabel='クラスター', ylabel='売り上げ')
    ax.legend(plot_dataset.index, fontsize = 25)
    plt.tick_params(labelsize=35)
    #plt.xticks(fontsize=10) #rotation=90
    plt.show()

In [ ]:
avgs_AC = avgs_A - avgs_C
avgs_BC = avgs_B - avgs_C
mean_AC = avgs_AC.loc['mean']
mean_BC = avgs_BC.loc['mean']
mean_AC = pd.DataFrame(mean_AC)
mean_AC = mean_AC.sort_values('mean')
mean_AC
mean_BC = pd.DataFrame(mean_BC)
mean_BC = mean_BC.sort_values('mean')
mean_BC

In [ ]:
not_injname_kakuninn[0]

In [ ]:
not_injname_kakuninn

#PCA
df_x_dim2 = []
for i in range(447):
    x_dim2_one = not_injname_kakuninn[i]
    x_dim2_one = pd.DataFrame(x_dim2_one)
    all_data_steps_one = all_data_steps[i]
    x_dim2_one = x_dim2_one.head(all_data_steps_one)
    x_dim2_one['pt_no'] = i
    df_x_dim2.append(x_dim2_one)
df_x_dim2[0]

concat_df_x_dim2 = pd.concat(df_x_dim2)
concat_df_x_dim2 = concat_df_x_dim2.reset_index(drop=True)

list_pt_no = concat_df_x_dim2['pt_no']
df_pt_no = pd.DataFrame(list_pt_no)

concat_df_x_dim2 = concat_df_x_dim2.drop(['pt_no'], axis = 1)


concat_df_x_dim2

#主成分分析で2次元にしてみる
from sklearn.decomposition import PCA
pca = PCA(n_components=16)

pca.fit(concat_df_x_dim2)
pca_x_dim2 = pca.transform(concat_df_x_dim2)

pca_x_dim2 = pd.DataFrame(pca_x_dim2)

#解体作業
pca_x_dim2 = pd.concat([pca_x_dim2, df_pt_no], axis = 1, join = 'inner')
pca_x_dim2
pca_dim2 = []
for x in range(447):
    pca_i_dim2 = pca_x_dim2[pca_x_dim2['pt_no'] == x]
    pca_i_dim2 = pca_i_dim2.drop(['pt_no'], axis = 1)
    pca_dim2.append(pca_i_dim2)
pca_dim2[0]

In [ ]:
concat_df_x_dim2.to_csv('concat_df.csv')

In [ ]:
concat_df_x_dim2
#hozonn
#concat_df_x_dim2.to_csv('pt_id/df_concat_not_injname.csv')

In [ ]:
#PCA16次元からTSNE2次元で次元削減してみる
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, perplexity = 30, random_state = 0)#perplexityの設定が大事らしい
vecs_list = tsne.fit_transform(pca_x_dim2)
vecs_list = pd.DataFrame(vecs_list)
vecs_list
#解体作業
tsne_x_dim2 = pd.concat([vecs_list, df_pt_no], axis = 1, join = 'inner')
tsne_x_dim2
tsne_dim2 = []
for x in range(447):
    tsne_i_dim2 = tsne_x_dim2[tsne_x_dim2['pt_no'] == x]
    tsne_i_dim2 = tsne_i_dim2.drop(['pt_no'], axis = 1)
    tsne_dim2.append(tsne_i_dim2)
tsne_dim2[0]

In [ ]:
vecs_list.to_csv('concat_df_pca16tsne2.csv')

In [ ]:
#TSNE2次元で次元削減してみる
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, perplexity = 30, random_state = 0)#perplexityの設定が大事らしい
vecs_list = tsne.fit_transform(concat_df_x_dim2)
vecs_list = pd.DataFrame(vecs_list)
vecs_list
#解体作業
tsne_x_dim2 = pd.concat([vecs_list, df_pt_no], axis = 1, join = 'inner')
tsne_x_dim2
tsne_dim2 = []
for x in range(447):
    tsne_i_dim2 = tsne_x_dim2[tsne_x_dim2['pt_no'] == x]
    tsne_i_dim2 = tsne_i_dim2.drop(['pt_no'], axis = 1)
    tsne_dim2.append(tsne_i_dim2)
tsne_dim2[0]

In [ ]:
df_pt_no.to_csv('df_pt_no.csv')

In [ ]:
vecs_list.to_csv('concat_df_tsne2.csv')

In [ ]:
#患者ごとver

#分散出す
list_tsne_dim2_mean = []
for j in range(447):
    tsne_dim2_one = tsne_dim2[j]

    #5回ずつの平均出す
    len_tsne_dim2 = len(tsne_dim2_one)//5
    if len_tsne_dim2 >=5:
        list_tsne_dim2_5mean = []
        for i in range(len_tsne_dim2):
            tsne_dim2_5mean_one = tsne_dim2_one.iloc[i*5:i*5 + 1, :]
            tsne_dim2_5mean_one = pd.DataFrame(tsne_dim2_5mean_one.mean()).T
            list_tsne_dim2_5mean.append(tsne_dim2_5mean_one)
        tsne_dim2_5mean = pd.concat(list_tsne_dim2_5mean)
        if len_tsne_dim2*5 == len(tsne_dim2_one):
            tsne_dim2_5mean = tsne_dim2_5mean
        else:
            tsne_dim2_lastmean = pd.DataFrame(tsne_dim2_one.iloc[len_tsne_dim2*5:len(tsne_dim2_one), :].mean()).T
            tsne_dim2_5mean = pd.concat([tsne_dim2_5mean, tsne_dim2_lastmean], axis = 0)
    else:
        tsne_dim2_5mean = pd.DataFrame(tsne_dim2_one.iloc[len_tsne_dim2*5:len(tsne_dim2_one), :].mean()).T
    
    list_tsne_dim2_mean.append(tsne_dim2_5mean)
list_tsne_dim2_mean[446]

In [ ]:
tsne_dim2_mean_one_s = []
for i in not_death_pt:
    tsne_dim2_mean_one = list_tsne_dim2_mean[i]
    tsne_dim2_mean_one_s.append(tsne_dim2_mean_one)
tsne_dim2_mean_one_not_death = pd.concat(tsne_dim2_mean_one_s)
tsne_dim2_mean_one_not_death

tsne_dim2_mean_not_death_var = tsne_dim2_mean_one_not_death.var()

tsne_dim2_mean_one_d = []
for i in death_pt:
    tsne_dim2_mean_one = list_tsne_dim2_mean[i]
    tsne_dim2_mean_one_d.append(tsne_dim2_mean_one)
tsne_dim2_mean_one_death = pd.concat(tsne_dim2_mean_one_d)
tsne_dim2_mean_one_death

tsne_dim2_mean_death_var = tsne_dim2_mean_one_death.var()

tsne_dim2_mean_not_death_var
tsne_dim2_mean_death_var

In [ ]:
#全患者さんver

#分散出す
tsne_dim2_mean = pd.concat(list_tsne_dim2_mean)
tsne_dim2_mean_var = tsne_dim2_mean.var()
tsne_dim2_mean_var

In [ ]:
tsne_dim2_mean_not_death_var

In [ ]:
plt.figure(figsize=(10, 10))

for p in not_death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    plt.plot(x, y, color = 'steelblue',lw = 0.1)
    #plt.scatter(x, y, color = 'b', s = 1)
    plt.scatter(z_1, z_2, color = 'steelblue', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))

for p in death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    plt.plot(x, y, color = 'indianred',lw = 0.1)
    #plt.scatter(x, y, color = 'r', s = 1)
    plt.scatter(z_1, z_2, color = 'indianred', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
    plt.axes().set_aspect('equal')
plt.xticks(color="None")
plt.yticks(color="None")
plt.grid(True)

plt.figure(figsize=(10, 10))

for p in not_death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'b',lw = 0.01)
    #plt.scatter(x, y, color = 'b', s = 1)
    plt.scatter(z_1, z_2, color = 'steelblue', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))

for p in death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'r',lw = 0.01)
    #plt.scatter(x, y, color = 'r', s = 1)
    plt.scatter(z_1, z_2, color = 'indianred', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.axes().set_aspect('equal')
plt.xticks(color="None")
plt.yticks(color="None")
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 10))

for p in not_death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'steelblue',lw = 0.01)
    plt.scatter(x, y, color = 'steelblue', s = 10, alpha = 0.1)
    plt.scatter(z_1, z_2, color = 'b', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.grid(True)
#plt.savefig("fig_save/notinjname_dim16_lr01_FFF/tsne_ds_plot_b_not_death.png")

plt.figure(figsize=(10, 10))
for p in death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'indianred',lw = 0.01)
    plt.scatter(x, y, color = 'indianred', s = 10, alpha = 0.1)
    plt.scatter(z_1, z_2, color = 'r', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.grid(True)

#plt.savefig("fig_save/notinjname_dim16_lr01_FFF/tsne_ds_plot_r_death.png")

In [ ]:
concat_df_x_dim2

In [ ]:
#最初からTSNE2次元で次元削減してみる
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, perplexity = 30, random_state = 0)#perplexityの設定が大事らしい
vecs_list = tsne.fit_transform(concat_df_x_dim2)
vecs_list = pd.DataFrame(vecs_list)
vecs_list
#解体作業
tsne_x_dim2 = pd.concat([vecs_list, df_pt_no], axis = 1, join = 'inner')
tsne_x_dim2
tsne_dim2 = []
for x in range(447):
    tsne_i_dim2 = tsne_x_dim2[tsne_x_dim2['pt_no'] == x]
    tsne_i_dim2 = tsne_i_dim2.drop(['pt_no'], axis = 1)
    tsne_dim2.append(tsne_i_dim2)
tsne_dim2[0]

In [ ]:
#患者ごとver

#分散出す
list_tsne_dim2_mean = []
for j in range(447):
    tsne_dim2_one = tsne_dim2[j]

    #5回ずつの平均出す
    len_tsne_dim2 = len(tsne_dim2_one)//5
    if len_tsne_dim2 >=5:
        list_tsne_dim2_5mean = []
        for i in range(len_tsne_dim2):
            tsne_dim2_5mean_one = tsne_dim2_one.iloc[i*5:i*5 + 1, :]
            tsne_dim2_5mean_one = pd.DataFrame(tsne_dim2_5mean_one.mean()).T
            list_tsne_dim2_5mean.append(tsne_dim2_5mean_one)
        tsne_dim2_5mean = pd.concat(list_tsne_dim2_5mean)
        if len_tsne_dim2*5 == len(tsne_dim2_one):
            tsne_dim2_5mean = tsne_dim2_5mean
        else:
            tsne_dim2_lastmean = pd.DataFrame(tsne_dim2_one.iloc[len_tsne_dim2*5:len(tsne_dim2_one), :].mean()).T
            tsne_dim2_5mean = pd.concat([tsne_dim2_5mean, tsne_dim2_lastmean], axis = 0)
    else:
        tsne_dim2_5mean = pd.DataFrame(tsne_dim2_one.iloc[len_tsne_dim2*5:len(tsne_dim2_one), :].mean()).T
    
    list_tsne_dim2_mean.append(tsne_dim2_5mean)
list_tsne_dim2_mean[446]

In [ ]:
tsne_dim2_mean_one_s = []
for i in not_death_pt:
    tsne_dim2_mean_one = list_tsne_dim2_mean[i]
    tsne_dim2_mean_one_s.append(tsne_dim2_mean_one)
tsne_dim2_mean_one_not_death = pd.concat(tsne_dim2_mean_one_s)
tsne_dim2_mean_one_not_death

tsne_dim2_mean_not_death_var = tsne_dim2_mean_one_not_death.var()

tsne_dim2_mean_one_d = []
for i in death_pt:
    tsne_dim2_mean_one = list_tsne_dim2_mean[i]
    tsne_dim2_mean_one_d.append(tsne_dim2_mean_one)
tsne_dim2_mean_one_death = pd.concat(tsne_dim2_mean_one_d)
tsne_dim2_mean_one_death

tsne_dim2_mean_death_var = tsne_dim2_mean_one_death.var()

tsne_dim2_mean_not_death_var
tsne_dim2_mean_death_var

In [ ]:
tsne_dim2_mean_not_death_var

In [ ]:
#全患者さんver

#分散出す
tsne_dim2_mean = pd.concat(list_tsne_dim2_mean)
tsne_dim2_mean_var = tsne_dim2_mean.var()
tsne_dim2_mean_var

In [ ]:
plt.figure(figsize=(10, 10))

for p in not_death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    plt.plot(x, y, color = 'steelblue',lw = 0.1)
    #plt.scatter(x, y, color = 'b', s = 1)
    plt.scatter(z_1, z_2, color = 'steelblue', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))

for p in death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    plt.plot(x, y, color = 'indianred',lw = 0.1)
    #plt.scatter(x, y, color = 'r', s = 1)
    plt.scatter(z_1, z_2, color = 'indianred', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.axes().set_aspect('equal')
plt.xticks(color="None")
plt.yticks(color="None")
plt.grid(True)

plt.figure(figsize=(10, 10))

for p in not_death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'b',lw = 0.01)
    #plt.scatter(x, y, color = 'b', s = 1)
    plt.scatter(z_1, z_2, color = 'steelblue', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))

for p in death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'r',lw = 0.01)
    #plt.scatter(x, y, color = 'r', s = 1)
    plt.scatter(z_1, z_2, color = 'indianred', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.axes().set_aspect('equal')
plt.xticks(color="None")
plt.yticks(color="None")
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 10))

for p in not_death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'steelblue',lw = 0.01)
    plt.scatter(x, y, color = 'steelblue', s = 10, alpha = 0.1)
    plt.scatter(z_1, z_2, color = 'b', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.grid(True)
#plt.savefig("fig_save/notinjname_dim16_lr01_FFF/tsne_ds_plot_b_not_death.png")

plt.figure(figsize=(10, 10))
for p in death_pt:
    x_dim2 = tsne_dim2[p]
    x = x_dim2[0]
    y = x_dim2[1]
    z_1 = x.tail(1)
    z_2 = y.tail(1)

    #plt.plot(x, y, color = 'indianred',lw = 0.01)
    plt.scatter(x, y, color = 'indianred', s = 10, alpha = 0.1)
    plt.scatter(z_1, z_2, color = 'r', s = 50)
    #plotにお名前つける
    #plt.annotate(p, (z_1,z_2))
plt.grid(True)

#plt.savefig("fig_save/notinjname_dim16_lr01_FFF/tsne_ds_plot_r_death.png")

In [ ]:
import pandas as pd
from sklearn import metrics
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import collections

#iris_dbscan = DBSCAN(eps=1, min_samples=10)
iris_dbscan = DBSCAN(eps=0.03, min_samples=5)
iris_dbscan.fit(tsne_x_dim2.values[:,0:2])
labels = iris_dbscan.labels_
print(labels)
lis_labels = list(labels)

tsne_x_dim2['cluster_DBSCAN'] = ['cluster' +str(x) for x in labels]
tsne_x_dim2
c = collections.Counter(tsne_x_dim2['cluster_DBSCAN'])
c_key = c.keys()
c_key = list(c_key)
c_key
#tsne_dim2_merge.to_csv('tsne_dim2_merge_DBSCAN', index = True)
tsne_x_dim2


In [ ]:
from sklearn.neighbors import NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=5)
nearest_neighbors.fit(pca_x_dim2.values[:,0:2])
distances, indices = nearest_neighbors.kneighbors(pca_x_dim2.values[:,0:2])
distances = np.sort(distances, axis=0)[:, 1]
print(distances)
plt.plot(distances)
plt.show()

In [ ]:
#sns.set_palette("Set2", 31)
#sns.set_palette("rainbow_r",30)
sns.pairplot(pca_x_dim2, hue = "cluster_DBSCAN",size = 10, hue_order = c_key,diag_kind="kde", aspect = 1.5)# diag_kind="kde",

In [ ]:
import seaborn as sns
plt.figure(figsize=(20, 20))
sns.scatterplot(x=tsne_x_dim2[0], y=tsne_x_dim2[1], hue = "cluster_DBSCAN", data = tsne_x_dim2)

遊び

In [ ]:
all_data_full = []
for i in range(447):
    x_dim2_one = all_data_df[i]
    x_dim2_one = pd.DataFrame(x_dim2_one)
    all_data_steps_one = all_data_steps[i]
    x_dim2_one = x_dim2_one.head(all_data_steps_one)
    all_data_full.append(x_dim2_one)
all_data_full[0].tail(1)

In [ ]:
tail_mean = []
for i in range(447):
    all_data_tail_one = all_data_full[i].tail(10)
    all_data_tail_one = all_data_tail_one.mean()
    all_data_tail_one = pd.DataFrame(all_data_tail_one)
    all_data_tail_one = all_data_tail_one.rename(columns = {0:i})
    all_data_tail_one = all_data_tail_one.iloc[:, :]
    tail_mean.append(all_data_tail_one)

In [ ]:
all_data_df[443].head(10)

In [ ]:
xxx_tconcat = pd.concat([tail_mean[251], tail_mean[81], tail_mean[257], tail_mean[326], tail_mean[301], tail_mean[205], tail_mean[107], tail_mean[51], tail_mean[61], tail_mean[261], tail_mean[18], tail_mean[139]], axis = 1)

In [ ]:
xxx_tconcat

In [ ]:
df = pd.concat([tail_mean[251], tail_mean[81]], axis = 1)
df_corr = df.corr

In [ ]:
#時系列感じるデータっぽいのみたい
#!pip install px
import plotly.express as px
import pandas as pd
df = all_data_full[0]
fig = px.line(df,x='Date', y=df.columns[0:108], title="all columns")

fig.show()

In [ ]:
#deathもきれいにしたい
df_PT_ID = pd.DataFrame()
df_PT_ID['PT_ID'] = list_PT_ID
all_data_death = pd.merge(df_PT_ID, df_death_opz, on = 'PT_ID', how = 'outer')
print(all_data_death)
#死亡の有無のところ、NaNに０、＋に１を入れる
death_isnull = all_data_death.isnull()
all_data_death = death_isnull.replace({True : 0, False : 1})
all_data_death

In [ ]:
all_data_df_ = []
for i in range(447):
    all_data_steps_one = all_data_steps[i]
    all_data_df_one = all_data_df[i].head(all_data_steps_one)
    all_data_df_.append(all_data_df_one)

In [ ]:
list_all_data_death = all_data_death['死亡の有無']
list_all_data_death = list(list_all_data_death)
list_all_data_death[0]

In [ ]:
tfm_all_data = []
for i in range(447):
    all_data_df_one = all_data_df_[i]

    all_data_df_one['Death'] = list_all_data_death[i]
    all_data_df_one['PT_ID'] = str(i)
    all_data_df_one.reset_index(inplace = True)
    all_data_df_one = all_data_df_one.rename(columns = {'index':'Date'})
    tfm_all_data.append(all_data_df_one)
tfm_all_data[0]
tfm_all_data = pd.concat(tfm_all_data)
tfm_all_data['Date'] = tfm_all_data.groupby('PT_ID').cumcount()

In [ ]:
tfm_all_data = pd.DataFrame(tfm_all_data).set_index(['PT_ID', 'Date'])

In [ ]:
tfm_all_data.head(75)

In [ ]:
X_columns = tfm_all_data.columns.to_list()
pdf = tfm_all_data
cur_t = range(447)

In [ ]:
#!pip install tdqm

In [ ]:
from tqdm.auto import tqdm


In [ ]:
lines = []
i = 0

lines += ['@problemName eutest']
lines += ['@timeStamps false']
lines += ['@missing false']
lines += ['@univariate false']
lines += ['@dimensions ' + str(len(X_columns))]
lines += ['@equalLength false']
lines += ['@classLabel true 0.0 1.0']
lines += ['@data']

for cur_PT in tqdm(pdf.reset_index()['PT_ID'].unique()):
    cur_PT_df = pdf.loc[[cur_PT], :]
    t_max = cur_PT_df.reset_index()['Date'].max()

    for cur_t in range(t_max+1):
        t_start = 0
        t_end = cur_t
        cur_df = cur_PT_df.query('@t_start <= Date <= @t_end')

        cur_items_str = []
        for cur_col in X_columns:
            cur_item_str = ','.join(cur_df[cur_col].astype('str').to_list())
            cur_items_str.append(cur_item_str)

        cur_ln = ':'.join(cur_items_str)

        cur_ln += ':'
        cur_ln += str(cur_df.loc[(cur_PT, cur_t), 'Death'])

        lines.append(cur_ln)

In [ ]:
#path = './tes_TRAIN.ts'
path = 'tes_TRAIN.ts'
with open(path, mode='w') as f:
    f.write('\n'.join(lines))

In [ ]:
#path = './tes_TEST.ts'
path = 'tes_TEST.ts'
with open(path, mode='w') as f:
    f.write('\n'.join(lines))